# Cleaning NYC high school data and exploring the fairness of the SAT  

The SAT, or Scholastic Aptitude Test, is a test that high school seniors in the U.S. take every year.  Colleges use the SAT to determine which students to admit.  High average SAT scores are usually indicative of a good school.  

New York City has published data on [student SAT scores](https://data.cityofnewyork.us/Education/2012-SAT-Results/f9bf-2cp4) by high school, along with additional related data sets:  

* SAT scores by school - SAT scores for each high school in New York City  
* High school directory - Various descriptive information for each high school in New York City (i.e. geographic coordinates)  
* Class size - Information on class size for each school  
* AP test results - Advanced Placement (AP) exam results for each high school (passing an optional AP exam in a particular subject can earn a student college credit in that subject)  
* Graduation outcomes - The percentage of students who graduated, and other outcome information  
* Demographics - Demographic information for each school  
* School survey - Surveys of parents, teachers, and students at each school  

New York City has a significant immigrant population and is very diverse, so comparing demographic factors such as race, income, and gender with SAT scores is a good way to determine whether the SAT is a fair test.  For example, if certain racial groups consistently perform better on the SAT, we would have some evidence that the SAT is unfair.  

I am going to clean the data and explore if the SAT is unfair to certain demographic groups or students in certain locations of NYC.  

This is the approach I am going to take...  

Here is a summary of my results...

# Read in the data  

We will read each comma separated value (csv) data file into a Pandas Dataframe and then store them in a dictionary named "data".  This will provide quick and efficient access to the different csv datasets by storing them all in one data structure.  

We will read in and explore the text (txt) survey data files later.

In [1]:
import pandas as pd
pd.options.display.max_columns = 100  # So dataframe columns will not be condensed

# Create a list of the csv data files
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

# Read each csv file into a dataframe and store in a dictionary
data = {}
for file in data_files:
    dataframe = pd.read_csv("../data/raw/{0}".format(file))  # apply the relative file locations
    key = file.replace(".csv", "")
    data[key] = dataframe
    
# View the first 5 rows of the "ap_2010" dataframe
data["ap_2010"].head()

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


# Exploring the SAT data  

We are interested in exploring relationships between the SAT data and the other datasets. We will look at the first 5 rows of the "sat_results" dataframe to see what we can learn on initial inspection.  

Referencing the "SAT_Results_Data_Dictionary" in the "references" folder of this project, each row provides the average SAT scores for each high school in NYC.  Each high school has a unique identification code called a "DBN" (district, borough, school number).  The scores are broken down into the 3 main sections of the test: Critical Reading, Math and Writing.  

We will explore any interesting correlations between this data and the other dataframes.

In [2]:
# View the first 5 rows of the "sat_results" dataframe
data["sat_results"].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


# Exploring the remaining data  

Similarly, we will look at the first 5 rows of each of the remaining dataframes to see what we can learn on initial inspection.  

### Exploring the "advanced placement" data  

Referencing the "AP_2010_Data_Dictionary" in the "references" folder of this project, this dataset provides advanced placement (AP) test results data for each NYC high school.  We can use this data to find any correlation between SAT results and AP results.

Each row represents various AP test results statistics for each NYC high school.  We can use the DBN column to join this data with the other dataframes.

In [3]:
# View the first 5 rows of the "ap_2010" dataframe
data["ap_2010"].head()

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39,49,10
1,01M450,EAST SIDE COMMUNITY HS,19,21,s
2,01M515,LOWER EASTSIDE PREP,24,26,24
3,01M539,"NEW EXPLORATIONS SCI,TECH,MATH",255,377,191
4,02M296,High School of Hospitality Management,s,s,s


### Exploring the "class size" data  

Referencing the "Class_Size_Data_Dictionary" in the "references" folder of this project, this dataset provides average class sizes for each school, by grade and program type for grades 5-9 and 9-12, aggregated by program type and core course.  We can use this data to find any correlation between SAT results and class size.

Each row represents a specific grade level and program type for that school.  We will have to filter for the rows that contain the grade levels and program types that are most suitable for our analysis.

In [4]:
# View the first 5 rows of the "class_size" dataframe
data["class_size"].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


There is also no DBN (district, borough, school number) column in this dataframe.  However, according to the "Class_Size_Data_Dictionary", the "CSD" column represents the district and the "school code" column represents the borough and school number.  This means we can create a DBN column by combining the "CSD" and "school code" columns and adding a leading 0 (to make it consistent with the other dataframes).  This will allow us to join this data with the other dataframes.

To confirm, we will take the DBN from the first row of the "sat_results" dataframe (above). That DBN is "01M292" and corresponds with the "HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES".  This means that in the "class_size" dataframe, the "CSD" should be "1" and the "school code" should be "M292" for that same school, which is confirmed in the next code cell. 

In [5]:
# View first 5 rows in "class_size" where "school code" is M292
data["class_size"][data["class_size"]["SCHOOL CODE"] == "M292"][["CSD", "SCHOOL CODE", "SCHOOL NAME"]].head()

,CSD,SCHOOL CODE,SCHOOL NAME
207,1,M292,Henry Street School for International Studies
208,1,M292,Henry Street School for International Studies
209,1,M292,Henry Street School for International Studies
210,1,M292,Henry Street School for International Studies
211,1,M292,Henry Street School for International Studies


### Exploring the "demographics" data  

Referencing the "Demographics_Data_Dictionary" in the "references" folder of this project, this dataset provides various demographic data for each NYC high school.  Some examples include: "percentage of English language learners" (ell_percent), "percentage of asian students" (asian_per) and "percentage of female students" (female_per).  This data will be instrumental in exploring if the SAT is unfair towards any particular demographic groups.

Each row represents a specific school year for that school.  We will have to filter for the rows that contain the school years that are most suitable for our analysis.  We can use the DBN column to join this data with the other dataframes.

In [6]:
# View the first 5 rows of the "demographics" dataframe
data["demographics"].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,grade3,grade4,grade5,grade6,grade7,grade8,grade9,grade10,grade11,grade12,ell_num,ell_percent,sped_num,sped_percent,ctt_num,selfcontained_num,asian_num,asian_per,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,38,52,29,38,NaN,NaN,NaN,NaN,NaN,NaN,36.0,12.8,57.0,20.3,25,9,10,3.6,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,34,42,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,15.6,55.0,22.6,19,15,18,7.4,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,38,47,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,19.9,60.0,23.0,20,14,16,6.1,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,34,39,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,19.0,62.0,24.6,21,17,16,6.3,75,29.8,149,59.1,7,2.8,149.0,59.1,103.0,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,30,24,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,19.2,46.0,22.1,14,14,16,7.7,67,32.2,118,56.7,6,2.9,124.0,59.6,84.0,40.4


### Exploring the "graduation" data  

Referencing the "Graduation_Data_Dictionary" in the "references" folder of this project, this dataset provides various graduation statistics for each NYC high school.  Some examples include: "percentage of students that graduated" (Total Grads - % of cohort) and "percentage of students that dropped out" (Dropped Out - % of Cohort).  We can use this data to find any correlation between SAT results and graduation rates.

Each row represents a specific cohort (the school year the SAT was administered) for that school.  We will have to filter for the rows that contain the cohort that is most suitable for our analysis.  We can use the DBN column to join this data with the other dataframes.

In [7]:
# View the first 5 rows of the "graduation" dataframe
data["graduation"].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,Advanced Regents - n,Advanced Regents - % of cohort,Advanced Regents - % of grads,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3%,17,30.9%,45.9%,0,0%,0%,17,30.9%,45.9%,20,36.4%,54.1%,15,27.3%,3,5.5%
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2%,27,42.2%,62.8%,0,0%,0%,27,42.2%,62.8%,16,25%,37.200000000000003%,9,14.1%,9,14.1%
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,0,0%,0%,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
4,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006 Aug,78,44,56.4%,37,47.4%,84.1%,0,0%,0%,37,47.4%,84.1%,7,9%,15.9%,15,19.2%,11,14.1%


### Exploring the "hs_directory" data  

Referencing the "HS_Directory_Data_Dictionary" in the "references" folder of this project, this dataset provides various descriptive information for each NYC high school.  Some examples include: "borough where the school is located" (boro) and "address/geographic coordinates" (Location 1).  We can use the location data to visualize the SAT results on a map of NYC.  This will allow us to easily see if there is any correlation between SAT results and specific areas of NYC.

Each row represents descriptive information for each NYC high school.  We can use the DBN column to join this data with the other dataframes.

In [9]:
# View the first 5 rows of the "hs_directory" dataframe
data["hs_directory"].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,"B41, B43, B44-SBS, B45, B48, B49, B69","2, 3, 4, 5, F, S to Botanic Garden ; B, Q to P...",883 Classon Avenue,Brooklyn,NY,11225,Bkmusicntheatre.com,399.0,Prospect Heights Educational Campus,NaN,Brooklyn School for Music & Theatre (BSMT) use...,We offer highly competitive positions in our D...,Spanish,"English Language and Composition, United State...",NaN,NaN,"Variety of clubs: Chess, The Step Team, Fashio...","Baseball, Basketball & JV Basketball, Cross Co...","Basketball, Cross Country, Indoor Track, Outdo...",NaN,NaN,F.Y.R.EZONE (Finding Your Rhythm thru Educatio...,NaN,NaN,"In 2002, Roundabout Theatre was selected by Ne...",One To World‘s Global Classroom connects New Y...,NaN,NaN,NaN,NaN,NaN,8:10 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Brooklyn students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67..."
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,"Bx13, Bx15, Bx17, Bx21, Bx35, Bx4, Bx41, Bx4A,...","2, 5 to Intervale Ave",1110 Boston Road,Bronx,NY,10456,www.hsvd.org,378.0,Morris Educational Campus,NaN,The High School for Violin and Dance (HSVD) is...,Freshmen take both violin and dance; College N...,Spanish,NaN,NaN,NaN,Advancement via Individual Determination (AVID...,"Baseball, Basketball & JV Basketball, Volleyball","Basketball, Softball, Volleyball",NaN,Morris Educational Campus Basketball and Volle...,McGraw Hill - Big Brother Big Sister,NaN,"Hostos Community College, Monroe College, Teac...",Bronx Arts Ensemble,buildOn,Print International,NaN,Bronx Cares,Our students are required to take four years o...,"Student Summer Orientation, Summer Internship ...",8:00 AM,3:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to Bronx students or residents who at...,Then to New York City residents who attend an ...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026..."
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,"Bx1, Bx11, Bx13, Bx18, Bx2, Bx3, Bx32, Bx35, Bx36","4 to Mt Eden Ave ; B, D to 170th St",1501 Jerome Avenue,Bronx,NY,10452,http://schools.nyc.gov/schoolportals/09/X327,543.0,DOE New Settlement Community Campus,NaN,At the Comprehensive Model School Project (CMS...,"After-school and Saturday Tutoring, Advisory, ...",Spanish,"Biology, Chemistry, United States History",NaN,NaN,"Choir, Gaming, Girls Club, Newspaper, Spanish,...",NaN,NaN,NaN,"As we expand, we plan to offer PSAL sports.",New Settlement Community Center,Montefiore Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Dress Code Required: white or baby blue button...,NaN,8:00 AM,4:00 PM,This school will provide students with disabil...,ESL,Functionally Accessible,1,Priority to continuing 8th graders,Then to Bronx students or residents who attend...,Then to New York City residents who attend an ...,Then to Bronx stud

# Reading in the survey data

In [ ]:
all_survey = pd.read_csv('survey_all.txt', delimiter='\t', encoding='windows-1252')
all_survey.head()

In [ ]:
d75_survey = pd.read_csv('survey_d75.txt', delimiter='\t', encoding='windows-1252')
d75_survey.head()

In [ ]:
survey = pd.concat([all_survey, d75_survey], axis=0)
print(all_survey.shape)
print(d75_survey.shape)
print(survey.shape)
survey.head()

In [ ]:
survey.columns

# 9. Cleaning Up the Surveys

In [ ]:
survey['DBN'] = survey['dbn']
survey[['dbn', 'DBN']]

In [ ]:
survey_cols = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", 
               "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", 
               "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", 
               "eng_tot_11", "aca_tot_11"]

In [ ]:
survey = survey.loc[:, survey_cols]
data['survey'] = survey
print(data['survey'].shape)
data['survey'].head()

In [ ]:
data['class_size'].head()

# 11. Inserting DBN Fields

In [ ]:
data['hs_directory']['DBN'] = data['hs_directory']['dbn']
data['hs_directory'][['dbn', 'DBN']].head()

In [ ]:
data['class_size'].head()

In [ ]:
def pad_csd(int):
    string = str(int)
    if len(string) == 2:
        return string
    else:
        return string.zfill(2)

In [ ]:
data['class_size']['padded_csd'] = data['class_size']['CSD'].apply(pad_csd)
data['class_size'][['CSD', 'padded_csd']].head()

In [ ]:
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']
data['class_size'][['DBN', 'padded_csd', 'SCHOOL CODE']].iloc[1000:1010]

In [ ]:
data['class_size'].head()

# 12. Combining the SAT Scores

In [ ]:
data['sat_results'].head()

In [ ]:
data['sat_results']['SAT Critical Reading Avg. Score'] = pd.to_numeric(
    data['sat_results']['SAT Critical Reading Avg. Score'], errors='coerce')

data['sat_results']['SAT Math Avg. Score'] = pd.to_numeric(
    data['sat_results']['SAT Math Avg. Score'], errors='coerce')

data['sat_results']['SAT Writing Avg. Score'] = pd.to_numeric(
    data['sat_results']['SAT Writing Avg. Score'], errors='coerce')

In [ ]:
data['sat_results'].dtypes

In [ ]:
data['sat_results']['sat_score'] = data[
    'sat_results']['SAT Critical Reading Avg. Score'] + data['sat_results']['SAT Math Avg. Score'] + data[
    'sat_results']['SAT Writing Avg. Score']

data['sat_results'].head()

# 13. Parsing Geographic Coordinates for Schools

In [ ]:
data['hs_directory'].head()

In [ ]:
import re
a = re.findall("\(.+\)", "1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)")
a

In [ ]:
b = a[0].split(', ')

b[0].replace('(', '').replace(')','')

In [ ]:
b[1].replace('(', '').replace(')','')

In [ ]:
def get_lat(string):
    coord = re.findall('\(.+\)', string)[0]
    lat = coord.split(', ')[0]
    lat_clean = lat.replace('(', '').replace(')','')
    
    return lat_clean

In [ ]:
get_lat("1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)")

In [ ]:
data['hs_directory']['lat'] = data['hs_directory']['Location 1'].apply(get_lat)
data['hs_directory'].head()

# 14. Extracting the Longitude

In [ ]:
def get_lon(string):
    coord = re.findall('\(.+\)', string)[0]
    lon = coord.split(', ')[1]
    lon_clean = lon.replace('(', '').replace(')','')
    
    return lon_clean

In [ ]:
get_lon("1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)")

In [ ]:
data['hs_directory']['lon'] = data['hs_directory']['Location 1'].apply(get_lon)
data['hs_directory'].head()

In [ ]:
data['hs_directory']['lat'] = pd.to_numeric(data['hs_directory']['lat'], errors='coerce')
data['hs_directory']['lon'] = pd.to_numeric(data['hs_directory']['lon'], errors='coerce')
data['hs_directory'].dtypes

In [ ]:
data['hs_directory'].head()

# Mission 2: Data Cleaning Walkthrough: Combining the Data

# 3. Condensing the Class Size Data Set

In [ ]:
class_size = data['class_size']
class_size.head()

In [ ]:
# View the unique values in the "GRADE" column of "class_size"
data["class_size"]["GRADE "].unique()  # The "GRADE" column name has a space in it

In [ ]:
class_size = class_size[class_size['GRADE '] == '09-12']
class_size = class_size[class_size['PROGRAM TYPE'] == 'GEN ED']
class_size.head()

In [ ]:
class_size['CORE SUBJECT (MS CORE and 9-12 ONLY)'].unique()

# 5. Computing Average Class Sizes

In [ ]:
import numpy as np

In [ ]:
class_size_group = class_size.groupby('DBN').agg(np.mean)
class_size = class_size_group
class_size

In [ ]:
class_size.reset_index(inplace=True)
class_size

In [ ]:
class_size.index

In [ ]:
class_size.columns

In [ ]:
data['class_size'] = class_size
data['class_size'].head()

# 7. Condensing the Demographics Data Set

In [ ]:
data['demographics'].head()

In [ ]:
data['demographics'].dtypes

In [ ]:
data['demographics'] = data['demographics'][data['demographics']['schoolyear'] == 20112012]
data['demographics'].head()

# 9. Condensing the Graduation Data Set

In [ ]:
data['graduation'].dtypes

In [ ]:
data['graduation'] = data['graduation'][data['graduation']['Cohort'] == '2006']
data['graduation'] = data['graduation'][data['graduation']['Demographic'] == 'Total Cohort']
data['graduation'].head()

# 10. Converting AP Test Scores

In [ ]:
data['ap_2010'].dtypes

In [ ]:
data['ap_2010'].head()

In [ ]:
data['ap_2010'].columns

In [ ]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    data['ap_2010'][col] = pd.to_numeric(data['ap_2010'][col], errors='coerce')

In [ ]:
data['ap_2010'].dtypes

# 12. Performing the Left Joins

In [ ]:
combined = data['sat_results']
combined.shape

In [ ]:
combined = combined.merge(data['ap_2010'], how='left', on='DBN')
combined.shape

In [ ]:
combined = combined.merge(data['graduation'], how='left', on='DBN')
combined.shape

In [ ]:
combined.head()

# 13. Performing the Inner Joins

In [ ]:
data.keys()

In [ ]:
join_keys = ['class_size', 'demographics', 'survey', 'hs_directory']

for key in join_keys:
    combined = combined.merge(data[key], how='inner', on='DBN')
    
combined.shape

In [ ]:
combined.head()

# 15. Filling in Missing Values

In [ ]:
type(combined.mean())

In [ ]:
means = combined.mean()
means

In [ ]:
combined.fillna(means, inplace=True)
combined

In [ ]:
combined.fillna(0, inplace=True)
print(combined.shape)
combined.head()

# 16. Adding a School Districut Column for Mapping

In [ ]:
'Sinbad'[:2]

In [ ]:
def first_two(string):
    return string[:2]

In [ ]:
combined['school_dist'] = combined['DBN'].apply(first_two)
combined[['DBN', 'school_dist']].head()

# Mission 3 Data Cleaning Walkthrough: Analyzing and Visualizing Data

# 3. Finding Correlations with the r Value

In [ ]:
correlations = combined.corr()
correlations

In [ ]:
correlations = correlations['sat_score']
correlations

In [ ]:
correlations['total_enrollment']

# 5. Plotting Enrollment with the Plot() Accessor

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
combined.plot(x='total_enrollment', y='sat_score', kind='scatter')
correlations['total_enrollment']

# 6. Exploring Schools With Low SAT Scores and Enrollment

In [ ]:
low_enrollment = combined[combined['total_enrollment'] < 1000]
low_enrollment = combined[combined['sat_score'] < 1000]
print(low_enrollment.shape)
low_enrollment[['School Name', 'total_enrollment', 'sat_score']]

# 7. Plotting Language Learning Percentage

In [ ]:
combined.plot(x='ell_percent', y='sat_score', kind='scatter')
correlations['ell_percent']

# 8. Mapping the Schools With Basemap

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

# 9. Mapping the Schools With Basemap

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = combined['lon'].tolist()
latitudes = combined['lat'].tolist()

m.scatter(longitudes, latitudes, s=20, zorder=2, latlon=True)

# 10. Plotting Out Statistics

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = combined['lon'].tolist()
latitudes = combined['lat'].tolist()

m.scatter(longitudes, latitudes, s=20, zorder=2, latlon=True, c=combined['ell_percent'], cmap='summer')

# 11. Calculating District Level Statistics

In [ ]:
districts = combined.groupby('school_dist').agg(np.mean)
districts

In [ ]:
districts.reset_index(inplace=True)
districts.head()

# 12. Plotting Percent Of English Learners by District

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = districts['lon'].tolist()
latitudes = districts['lat'].tolist()

m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, c=districts['ell_percent'], cmap='summer')

# Guided Project: Analyzing NYC High School Data

# 1. Introduction

In [ ]:
survey_fields_with_SAT = [
    "DBN",
    'sat_score',
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_11", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
    ]

In [ ]:
combined.loc[:,survey_fields_with_SAT]

In [ ]:
corr_survey_sat = combined.loc[:,survey_fields_with_SAT].corr()
corr_survey_sat = corr_survey_sat['sat_score'][1:]
corr_survey_sat

In [ ]:
corr_survey_sat.plot.barh()

### Findings

Mine: Academic expectations and safety and respect scores have strong r-values with SAT scores. Makes sense, hard to learn in an unsafe environment, and I'd expect high achieveing students with high expectations of a school to perform well academically at that school.

Solution: There are high correlations between N_s, N_t, N_p and sat_score. Since these columns are correlated with total_enrollment, it makes sense that they would be high.

It is more interesting that rr_s, the student response rate, or the percentage of students that completed the survey, correlates with sat_score. This might make sense because students who are more likely to fill out surveys may be more likely to also be doing well academically.

How students and teachers percieved safety (saf_t_11 and saf_s_11) correlate with sat_score. This make sense, as it's hard to teach or learn in an unsafe environment.

The last interesting correlation is the aca_s_11, which indicates how the student perceives academic standards, correlates with sat_score, but this is not true for aca_t_11, how teachers perceive academic standards, or aca_p_11, how parents perceive academic standards.

# 2. Exploring Safety and SAT Scores

In [ ]:
combined.plot.scatter(x='saf_s_11', y='sat_score')

### Findings

Mine: Doesn't appear to be a strong linear correlations, weak at best.

Solution: There appears to be a correlation between SAT scores and safety, although it isn't that strong. It looks like there are a few schools with extremely high SAT scores and high safety scores. There are a few schools with low safety scores and low SAT scores. No school with a safety score lower than 6.5 has an average SAT score higher than 1500 or so.

In [ ]:
combined.head()

In [ ]:
school_dist = combined.groupby('school_dist').agg(np.mean)
school_dist.reset_index(inplace=True)
school_dist

In [ ]:
m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='l'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)

longitudes = school_dist['lon'].tolist()
latitudes = school_dist['lat'].tolist()

m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, c=school_dist['saf_s_11'], cmap='summer')

### Findings

Mine: Safest schools are in Manhattan as expected. With exception of a few, unsafest schools are in Brokklyn, Queens and the Bronx

Solution: It looks like Upper Manhattan and parts of Queens and the Bronx tend to have lower safety scores, whereas Brooklyn has high safety scores.

# 3. Exploring Race and SAT Scores

In [ ]:
race_cols = ['sat_score', 'white_per', 'asian_per', 'black_per', 'hispanic_per']

In [ ]:
race_sat = combined[race_cols].corr()['sat_score'][1:]
race_sat

In [ ]:
race_sat.plot.barh()

### Findings

Mine: Asian and White are strongly positively correlated. Hispanic and Black are strongly negatively correlated.  Possibly suggests racial bias in test, or indicative of lack of funding for schools in areas with high % of hispanic and black students.

Solution: It looks like a higher percentage of white or asian students at a school correlates positively with sat score, whereas a higher percentage of black or hispanic students correlates negatively with sat score. This may be due to a lack of funding for schools in certain areas, which are more likely to have a higher percentage of black or hispanic students.

In [ ]:
combined.plot.scatter('hispanic_per', 'sat_score')

### Findings

Mine: appears to be negative correlation, although not that strong. Schools with greater than approx 25% hispanic don't have an SAT score above around 1500

In [ ]:
combined.head()

In [ ]:
combined[combined['hispanic_per'] > 95]['SCHOOL NAME']

### Findings

Solution: The schools listed above appear to primarily be geared towards recent immigrants to the US. These schools have a lot of students who are learning English, which would explain the lower SAT scores.

In [ ]:
hisp_10_1800 = combined[combined['hispanic_per'] < 10]
hisp_10_1800[hisp_10_1800['sat_score'] > 1800]['SCHOOL NAME']

### Findings

Mine: all appear to be science and technical schools, suggesting higher acceptance requirements, and hence higher achieving students

Solution: Many of the schools above appear to be specialized science and technology schools that receive extra funding, and only admit students who pass an entrance exam. This doesn't explain the low hispanic_per, but it does explain why their students tend to do better on the SAT -- they are students from all over New York City who did well on a standardized test.

# 4. Exploring Gender and SAT Scores

In [ ]:
sex_cols = ['sat_score', 'male_per', 'female_per']

In [ ]:
sex_sat = combined[sex_cols].corr()['sat_score'][1:]
sex_sat

In [ ]:
sex_sat.plot.bar()

### Findings

Mine: females positive corr, males negative corr, but neither are strong.

Solution: In the plot above, we can see that a high percentage of females at a school positively correlates with SAT score, whereas a high percentage of males at a school negatively correlates with SAT score. Neither correlation is extremely strong.

In [ ]:
combined.plot.scatter('female_per', 'sat_score')

### Findings

Mine: don't see any strong linear corr.  There's high and low sat scores with high and low female_per

Solution: Based on the scatterplot, there doesn't seem to be any real correlation between sat_score and female_per. However, there is a cluster of schools with a high percentage of females (60 to 80), and high SAT scores.

In [ ]:
fem_60_1700 = combined[combined['female_per'] > 60]
fem_60_1700[fem_60['sat_score'] > 1700]['SCHOOL NAME']

### Findings

Solution: These schools appears to be very selective liberal arts schools that have high academic standards

# 5. Exploring AP Scores vs. SAT Scores

In [ ]:
combined['ap_per'] = combined['AP Test Takers '] / combined['total_enrollment']
combined['ap_per']

In [ ]:
combined.plot.scatter('ap_per', 'sat_score')

### Findings

Mine: some corr but not strong. There is a cluster of schools with ap_per > .4 with high SAT, but also a bunch with very high ap_per with SAT no greater than 1200

Solution: It looks like there is a relationship between the percentage of students in a school who take the AP exam, and their average SAT scores. It's not an extremely strong correlation, though.